Calculate top 5 products read from parquet and 
result written to top5_product

In [2]:
%%pyspark
df = spark.read.load('abfss://asa-kz@asadatalake19hvmew.dfs.core.windows.net/top-products/*.parquet', format='parquet')
display(df.limit(10))

StatementMeta(SparkPool01, 28, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, c45eb09b-11d8-45f4-8d46-dd1ff37062e7)

In [3]:
topPurchases = df.select(
    "UserId", 
    "ProductId",
    "ItemsPurchasedLast12Months",
    "IsTopProduct",
    "IsPreferredProduct")

# create temp  view top_purchases
topPurchases.createOrReplaceTempView("top_purchases")

topPurchases.show(11)

StatementMeta(SparkPool01, 28, 3, Finished, Available)

+------+---------+--------------------------+------------+------------------+
|UserId|ProductId|ItemsPurchasedLast12Months|IsTopProduct|IsPreferredProduct|
+------+---------+--------------------------+------------+------------------+
|   148|     2717|                      null|       false|              true|
|   148|     4002|                      null|       false|              true|
|   148|     1716|                      null|       false|              true|
|   148|     4520|                      null|       false|              true|
|   148|      951|                      null|       false|              true|
|   148|     1817|                      null|       false|              true|
|   148|     2717|                      null|       false|              true|
|   148|     4002|                      null|       false|              true|
|   148|     1716|                      null|       false|              true|
|   148|     4520|                      null|       false|      

In [5]:
%%sql
select * from top_purchases LIMIT 11

StatementMeta(SparkPool01, 28, 5, Finished, Available)

<Spark SQL result set with 11 rows and 5 fields>

In [6]:
from pyspark.sql.functions import *

topPreferredProducts = (topPurchases
    .filter( col("IsTopProduct") == True)
    .filter( col("IsPreferredProduct") == True)
    .orderBy( col("ItemsPurchasedLast12Months").desc() ))

topPreferredProducts.show(11)

StatementMeta(SparkPool01, 28, 6, Finished, Available)

+------+---------+--------------------------+------------+------------------+
|UserId|ProductId|ItemsPurchasedLast12Months|IsTopProduct|IsPreferredProduct|
+------+---------+--------------------------+------------+------------------+
| 84445|     1001|                        99|        true|              true|
| 90804|     1709|                        99|        true|              true|
| 90804|     1709|                        99|        true|              true|
| 90804|     1709|                        99|        true|              true|
| 90804|     1709|                        99|        true|              true|
| 89537|     1473|                        99|        true|              true|
| 90804|     1709|                        99|        true|              true|
| 89537|     1473|                        99|        true|              true|
| 90804|     1709|                        99|        true|              true|
| 89537|     1473|                        99|        true|      

Additional transofrmation in SQL 
row)numbser 
where

In [7]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW top_5_products
AS
    select 
        UserId,
        ProductId, 
        ItemsPurchasedLast12Months
    from (
            select *,
                row_number() over (partition by UserId order by ItemsPurchasedLast12Months desc) as seqnum
            from top_purchases
        ) a
    where 
        seqnum <= 5 and 
        IsTopProduct == true and 
        IsPreferredProduct = true
    order by
        a.UserId

StatementMeta(SparkPool01, 28, 7, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
top5Products = sqlContext.table("top_5_products")

top5Products.show(11)

StatementMeta(SparkPool01, 28, 8, Finished, Available)

+------+---------+--------------------------+
|UserId|ProductId|ItemsPurchasedLast12Months|
+------+---------+--------------------------+
| 80000|     2069|                        93|
| 80000|     2069|                        93|
| 80000|     2069|                        93|
| 80000|     2069|                        93|
| 80000|     2069|                        93|
| 80001|     1812|                        93|
| 80001|     1812|                        93|
| 80001|     1812|                        93|
| 80001|     1812|                        93|
| 80001|     1812|                        93|
| 80002|     4987|                        88|
+------+---------+--------------------------+
only showing top 11 rows



In [9]:
print('topPreferredProducts(IsTopProduct=T,IsPreferredProduct=T): ', topPreferredProducts.count(), ', top5Products (filter: ', top5Products.count())

StatementMeta(SparkPool01, 28, 9, Finished, Available)

topPreferredProducts(IsTopProduct=T,IsPreferredProduct=T):  997817 , top5Products (filter:  85015


In [10]:
top5ProductsOverall = (top5Products.select("ProductId","ItemsPurchasedLast12Months")
    .groupBy("ProductId")
    .agg( sum("ItemsPurchasedLast12Months").alias("Total") )
    .orderBy( col("Total").desc() )
    .limit(5))

top5ProductsOverall.show()

StatementMeta(SparkPool01, 28, 10, Finished, Available)

+---------+-----+
|ProductId|Total|
+---------+-----+
|      347| 4523|
|     4833| 4314|
|     3459| 4233|
|     2486| 4135|
|     2107| 4113|
+---------+-----+



In [11]:
import uuid

# Generate random GUID
runId = uuid.uuid4()

StatementMeta(SparkPool01, 28, 11, Finished, Available)

Write calculated results  to datalake storage as parquet in top5-products dir 

In [12]:
%%pyspark

top5ProductsOverall.write.parquet('abfss://asa-kz@asadatalake19hvmew.dfs.core.windows.net/top5-products/' + str(runId) + '.parquet')

StatementMeta(SparkPool01, 28, 12, Finished, Available)